In [1]:
from IPython.display import Markdown, display
from openai import OpenAI
import json
import os
from time import sleep

In [14]:
client = OpenAI()
small_dataset_model = "ft:gpt-4o-mini-2024-07-18:researcher-access-university-of-copenhagen:random-small-dataset:AYDdDFb6"

In [15]:
model_used= small_dataset_model

In [16]:
# Create the system_prompt
example_simple = """User input: "חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"
Expected output: ["חזון ישעיהו בןאמוץ","אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"]"""

example_complex = """User input: "יביא יהוה עליך ועלעמך ועלבית אביך ימים אשר לאבאו למיום סוראפרים מעל יהודה את מלך אשור פ"
Expected output: ["יביא יהוה עליך ועלעמך ועלבית אביך ימים את מלך אשור פ" ,"אשר לאבאו למיום" ,"סוראפרים מעל יהודה"]"""


SYSTEM_PROMPT = f"Identify clauses in this Biblical Hebrew verse and return a JSON list containing the clauses, as shown in the following examples. Do not write anything else than the JSON list in your output. Example 1 {example_simple} Example 2 {example_complex}"

In [17]:
SYSTEM_PROMPT

'Identify clauses in this Biblical Hebrew verse and return a JSON list containing the clauses, as shown in the following examples. Do not write anything else than the JSON list in your output. Example 1 User input: "חזון ישעיהו בןאמוץ אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"\nExpected output: ["חזון ישעיהו בןאמוץ","אשר חזה עליהודה וירושלם בימי עזיהו יותם אחז יחזקיהו מלכי יהודה"] Example 2 User input: "יביא יהוה עליך ועלעמך ועלבית אביך ימים אשר לאבאו למיום סוראפרים מעל יהודה את מלך אשור פ"\nExpected output: ["יביא יהוה עליך ועלעמך ועלבית אביך ימים את מלך אשור פ" ,"אשר לאבאו למיום" ,"סוראפרים מעל יהודה"]'

In [22]:
# Functions

def parse_verse (verse):
    """Send a request to the chosen OpenAI model to parse a Biblical Hebrew verse into clauses."""
    try:
        response = client.chat.completions.create(
            model=model_used,
            messages=[
                {"role": "system", "content": SYSTEM_PROMPT},
                {"role": "user", "content": f"{verse}"}
            ]
        )
        # Extract and return the response
        
        return response.choices[0].message
    except Exception as e:
        print(f"Error processing verse: {verse}")
        print(e)
        return None


def process_file(input_file, output_file):
    """Read a JSONL file containing verses, process them and save results to another JSONL file."""
    with open(input_file, "r", encoding="utf-8") as infile, open(output_file, "w", encoding="utf_8") as outfile:
        for line in infile:
            data=json.loads(line)
            verse = data["messages"][1]["content"]
            #print(verse)
            while True:
                result = parse_verse(verse).content
                if result:
                    break
                else:
                    sleep(5)
                    print("Failed to call OpenAI. Retrying.")
            if result:
                output = {"verse": verse, "parsed_clauses": result}
                outfile.write(json.dumps(output, ensure_ascii=False) + "\n")
            
            

# Input and output files
input_jsonl_file = "data/fine_tuning_datasets/trial_1/validation_small.jsonl"
output_jsonl_file = "data/fine_tuning_datasets/trial_1/model_outputs/output_validation_small.jsonl"

In [23]:
parse_verse("Tralala")

ChatCompletionMessage(content='["Tralala"]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None)

In [24]:
process_file(input_jsonl_file, output_jsonl_file)